In [1]:
import numpy as np # 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
UCSDB=pd.read_excel('./uncleanData.xlsx',index_col=None, thousands=',')
print( UCSDB.columns )

Index(['Name of Satellite, Alternate Names',
       'Current Official Name of Satellite', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', 'Dry Mass (kg.)', 'Power (watts)',
       'Date of Launch', 'Expected Lifetime (yrs.)', 'Contractor',
       'Country of Contractor', 'Launch Site', 'Launch Vehicle',
       'COSPAR Number', 'NORAD Number', 'Comments', 'Unnamed: 28',
       'Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2',
       'Source.3', 'Source.4', 'Source.5', 'Source.6', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49',
  

In [3]:
UCSDB.drop(list(UCSDB.filter(regex='Source|Unnamed')), axis=1, inplace=True)
UCSDB.drop(columns=['Comments'], inplace=True)
UCSDB.convert_dtypes()
UCSDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 27 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Name of Satellite, Alternate Names  7560 non-null   object 
 1   Current Official Name of Satellite  7560 non-null   object 
 2   Country/Org of UN Registry          7559 non-null   object 
 3   Country of Operator/Owner           7560 non-null   object 
 4   Operator/Owner                      7560 non-null   object 
 5   Users                               7560 non-null   object 
 6   Purpose                             7560 non-null   object 
 7   Detailed Purpose                    1254 non-null   object 
 8   Class of Orbit                      7560 non-null   object 
 9   Type of Orbit                       6909 non-null   object 
 10  Longitude of GEO (degrees)          7557 non-null   float64
 11  Perigee (km)                        7553 no

In [4]:
UCSDB['Class of Orbit'] = UCSDB['Class of Orbit'].str.upper().astype('category')
UCSDB['Type of Orbit'] = UCSDB['Type of Orbit'].str.upper().astype('category')
UCSDB[['Class of Orbit', 'Type of Orbit']].value_counts()

Class of Orbit  Type of Orbit             
LEO             NON-POLAR INCLINED            3942
                SUN-SYNCHRONOUS               1688
                POLAR                         1096
MEO             NON-POLAR INCLINED              98
ELLIPTICAL      MOLNIYA                         23
MEO             EQUATORIAL                      20
LEO             EQUATORIAL                      18
ELLIPTICAL      DEEP HIGHLY ECCENTRIC            9
LEO             ELLIPTICAL                       5
ELLIPTICAL      SUN-SYNCHRONOUS                  4
LEO             SUN-SYNCHRONOUS NEAR POLAR       2
ELLIPTICAL      NON-POLAR INCLINED               2
                CISLUNAR                         1
LEO             RETROGRADE                       1
Name: count, dtype: int64

In [5]:
replace_map = {
	'Commercial/Government': 'Government/Commercial',
	'Commercial/Military': 'Military/Commercial',
	'Civil/Military': 'Military/Civil',
	'Civil/Commercial': 'Commercial/Civil',
	'Civil/Government': 'Government/Civil',
	'Government/Military': 'Military/Government'
}
UCSDB['Users'] = UCSDB['Users'].str.strip()

UCSDB['Users'] = UCSDB['Users'].replace(replace_map)
UCSDB['Users'].value_counts()

Users
Commercial                        6081
Government                         559
Military                           458
Civil                              160
Government/Commercial               98
Military/Commercial                 84
Military/Government                 60
Government/Civil                    44
Military/Civil                      10
Commercial/Civil                     5
Government/Commercial/Military       1
Name: count, dtype: int64

In [6]:
UCSDB['Purpose'] = UCSDB['Purpose'].str.upper().astype('string')
UCSDB['Detailed Purpose'] = UCSDB['Detailed Purpose'].str.upper().astype('string')
UCSDB['Operator/Owner'] = UCSDB['Operator/Owner'].str.strip().astype('string')

In [7]:
UCSDB[['Expected Lifetime (yrs.)','Dry Mass (kg.)', 'Launch Mass (kg.)', 'Eccentricity', 'Inclination (degrees)','Period (minutes)', 'Power (watts)']]=\
    UCSDB[['Expected Lifetime (yrs.)','Dry Mass (kg.)', 'Launch Mass (kg.)', 'Eccentricity', 'Inclination (degrees)','Period (minutes)', 'Power (watts)']]\
                                                                                            .apply(pd.to_numeric,errors='coerce')


In [8]:
parsed_date_of_launch = pd.to_datetime(UCSDB['Date of Launch'], errors='coerce').sort_values()
# determine if there are any data points not correctly formatted
print( parsed_date_of_launch.loc[ np.isnat(parsed_date_of_launch) ] )


240    NaT
349    NaT
7186   NaT
Name: Date of Launch, dtype: datetime64[ns]


In [9]:
UCSDB.loc[ np.isnat(parsed_date_of_launch), ['Name of Satellite, Alternate Names','NORAD Number','Date of Launch'] ]

,"Name of Satellite, Alternate Names",NORAD Number,Date of Launch
240,BlackSky Global 5,55983,NaN
349,Cicero-8 (Community Initiative for Cellular Ea...,43737,11/29/018
7186,Tianmu-1 01,55134,1/9//2023


In [10]:
UCSDB.loc[ np.isnat(parsed_date_of_launch), 'Date of Launch'] = pd.to_datetime(["03/24/2023", "11/29/2018", "09/01/2023"])
UCSDB.loc[ np.isnat(parsed_date_of_launch) ]['Date of Launch']

240     2023-03-24 00:00:00
349     2018-11-29 00:00:00
7186    2023-09-01 00:00:00
Name: Date of Launch, dtype: object

In [11]:
UCSDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 27 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   Name of Satellite, Alternate Names  7560 non-null   object  
 1   Current Official Name of Satellite  7560 non-null   object  
 2   Country/Org of UN Registry          7559 non-null   object  
 3   Country of Operator/Owner           7560 non-null   object  
 4   Operator/Owner                      7560 non-null   string  
 5   Users                               7560 non-null   object  
 6   Purpose                             7560 non-null   string  
 7   Detailed Purpose                    1254 non-null   string  
 8   Class of Orbit                      7560 non-null   category
 9   Type of Orbit                       6909 non-null   category
 10  Longitude of GEO (degrees)          7557 non-null   float64 
 11  Perigee (km)                  

In [12]:
UCSDB['Country/Org of UN Registry'] = UCSDB['Country/Org of UN Registry'].apply(
    lambda x: 'NR' if pd.notnull(x) and str(x).strip().startswith('NR') else x
)
UCSDB['Country/Org of UN Registry'].value_counts()

Country/Org of UN Registry
USA                     4991
NR                       868
United Kingdom           655
China                    319
Russia                   166
France                    82
Japan                     71
India                     49
Luxembourg                39
Canada                    34
Uruguay                   33
Germany                   32
ESA                       24
Finland                   19
Brazil                    17
Australia                 14
South Korea               13
United Arab Emirates      11
Spain                     11
Italy                     10
Saudi Arabia              10
Norway                     8
Turkey                     7
Mexico                     6
Indonesia                  5
Lithuania                  5
Denmark                    5
Belgium                    5
Algeria                    5
EUMETSAT                   5
Argentina                  3
Pakistan                   3
South Africa               3
Azerbaijan      

In [13]:
UCSDB['Launch Site'] = UCSDB['Launch Site'].str.strip()

In [14]:
launch_site_to_country = {
	'Cape Canaveral': 'United States',
	'Vandenberg AFB': 'United States',
	'Vandeberg AFB': 'United States',  # typo variant
	'Wallops Island Flight Facility': 'United States',
	'Mid-Atlantic Regional Spaceport/Wallops Island': 'United States',
	'Kodiak Island': 'United States',
	'Kodiak Launch Complex': 'United States',
	'Antares': 'United States',  # should check this row — probably Orbital ATK's pad, USA
	'Orbital ATK L-1011': 'United States',
	'Cygnus': 'United States',
	'International Space Station - Cygnus': 'International',
	'International Space Station': 'International',
	'Stargazer L-1011': 'United States',
	'Rocket Lab Launch Complex 1': 'New Zealand',
	'Rocket Lab Launch Complex 1B': 'New Zealand',
	'Baikonur Cosmodrome': 'Kazakhstan',
	'Plesetsk Cosmodrome': 'Russia',
	'Vostochny Cosmodrome': 'Russia',
	'Svobodny Cosmodrome': 'Russia',
	'Dombarovsky Air Base': 'Russia',
	'Guiana Space Center': 'French Guiana',
	'Satish Dhawan Space Centre': 'India',
	'Satish Dhawan Space Center': 'India',
	'Jiuquan Satellite Launch Center': 'China',
	'Taiyuan Launch Center': 'China',
	'Taiyan Launch Center': 'China',  # typo variant
	'Xichang Satellite Launch Center': 'China',
	'Wenchang Space Center': 'China',
	'Wenchang Satellite Launch Center': 'China',
	'Yellow Sea Launch Platform': 'China',
	'Tanegashima Space Center': 'Japan',
	'Uchinoura Space Center': 'Japan',
	'Naro Space Center': 'South Korea',
	'Palmachim Launch Complex': 'Israel',
	'Kwajalein Island': 'Marshall Islands',
	'Sea Launch Odyssey': 'International Waters',
	'Virgin Orbit': 'United States',  # launched from air, USA-based
	'FANTM-RAiL [Xtenti]': 'United States',
	'FANTM-RAiL (Xtenti)': 'United States',
	'Shahroud Missile Range': 'Iran'
}
UCSDB['Launch Country'] = UCSDB['Launch Site'].map(launch_site_to_country)
print(UCSDB[UCSDB['Launch Country'].isnull()]['Launch Site'].unique())


[]


In [15]:
UCSDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 28 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   Name of Satellite, Alternate Names  7560 non-null   object  
 1   Current Official Name of Satellite  7560 non-null   object  
 2   Country/Org of UN Registry          7559 non-null   object  
 3   Country of Operator/Owner           7560 non-null   object  
 4   Operator/Owner                      7560 non-null   string  
 5   Users                               7560 non-null   object  
 6   Purpose                             7560 non-null   string  
 7   Detailed Purpose                    1254 non-null   string  
 8   Class of Orbit                      7560 non-null   category
 9   Type of Orbit                       6909 non-null   category
 10  Longitude of GEO (degrees)          7557 non-null   float64 
 11  Perigee (km)                  

In [16]:
def normalize(val):
    # If there is a slash, take the second part
    if '/' in val:
        val = val.split('/')[1]
    # Remove trailing numbers
    val = ''.join(filter(lambda x: not x.isdigit(), val))
    # Trim spaces
    val = val.strip()
    # Normalize Spacex variations
    if val.lower() == 'spacex':
        val = 'SpaceX'
    return val
UCSDB['Operator/Owner'] = UCSDB['Operator/Owner'].apply(normalize)


In [25]:
def normalize2(val):
    val_lower = val.lower()
    if 'nasa' in val_lower or 'national aeronautics and space administration' in val_lower:
        val = 'NASA'
    return val
UCSDB['Operator/Owner'] = UCSDB['Operator/Owner'].apply(normalize2)


In [26]:
UCSDB.to_csv('cleanData.csv', index=False)
